In [1]:
import os
import warnings

from db_queries import get_location_metadata
import pandas as pd

from covid_model_deaths import runner
from covid_model_deaths.deaths_io import InputsContext, MEASURES, Checkpoint
from covid_model_deaths.globals import COLUMNS

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99
warnings.simplefilter('ignore')

RUN_TYPE = 'dev'
DATA_VERSION = 'best'
DATESTAMP_LABEL = '2020_05_03_US_boundary'

PEAK_FILE = '/ihme/covid-19/deaths/mobility_inputs/2020_04_20/peak_locs_april20_.csv'
PEAK_DURATION_FILE = None
R0_FILE = None
LOCATION_SET_VERSION = 655
r0_locs = []

CODE_DIR = os.path.abspath('../src/covid_model_deaths')
OUTPUT_DIR = f'/ihme/covid-19/deaths/{RUN_TYPE}/{DATESTAMP_LABEL}'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
inputs = InputsContext(f'/ihme/covid-19/model-inputs/{DATA_VERSION}')
checkpoint = Checkpoint(OUTPUT_DIR)

print(f'Writing to {OUTPUT_DIR}')
print(CODE_DIR)
print(checkpoint)

raw_draw_path = f'{OUTPUT_DIR}/state_data.csv'
average_draw_path = f'{OUTPUT_DIR}/past_avg_state_data.csv'
yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_05_01_US_newmod/state_data.csv'
before_yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_29_US/state_data.csv'


def filter_data(data: pd.DataFrame) -> pd.DataFrame:
    """Remove bad or problematic data locations and dates."""
    # dropping data from May 1 for Mississippi for Lazio and Mississippi
    mississippi_lazio = data['location_id'].isin([547, 35506])
    mississippi_lazio_spike = mississippi_lazio & (data['Date'] == pd.Timestamp('2020-05-01'))
    data = data.loc[~mississippi_lazio_spike].reset_index(drop=True)
    
    return data

def get_locations(location_set_version_id):
    loc_df = get_location_metadata(location_set_id=111,
                                   location_set_version_id=location_set_version_id)
    most_detailed = loc_df['most_detailed'] == 1
    us = loc_df['path_to_top_parent'].str.startswith('102,')
    keep_columns = ['location_id', 'location_ascii_name', 'parent_id', 'level', 'most_detailed']

    us_df = loc_df.loc[most_detailed & us, keep_columns]
    us_df = us_df.rename(columns={'location_ascii_name': 'Location'})

    # Add parents
    loc_df = loc_df[['location_id', 'location_ascii_name']]
    loc_df = loc_df.rename(columns={'location_id':'parent_id',
                                    'location_ascii_name':'Country/Region'})

    us_df = us_df.merge(loc_df)
    us_df = us_df.loc[:, ['location_id', 'Location', 'Country/Region', 'level']]
    return us_df


2020-05-04 17:30:06.703 | DEBUG    | covid_model_deaths.deaths_io.checkpoint:_setup_checkpoint_dir:45 - Making checkpoint directory at /ihme/covid-19/deaths/dev/2020_05_03_US_boundary/checkpoint


Writing to /ihme/covid-19/deaths/dev/2020_05_03_US_boundary
/ihme/code/rmbarber/covid-model-deaths/src/covid_model_deaths
Checkpoint(/ihme/covid-19/deaths/dev/2020_05_03_US_boundary/checkpoint)


## read full (unrestricted) set from snapshot

In [2]:
loc_df = get_locations(LOCATION_SET_VERSION)
input_full_df = filter_data(inputs.load(MEASURES.full_data))
input_death_df = filter_data(inputs.load(MEASURES.deaths))
input_age_pop_df = inputs.load(MEASURES.age_pop)
input_age_death_df = inputs.load(MEASURES.age_death)

# Save pops for Bobby.
inputs.load(MEASURES.us_pops).to_csv(f'{OUTPUT_DIR}/pops.csv', index=False)

checkpoint.write('location', loc_df)
checkpoint.write('full_data', input_full_df)
checkpoint.write('deaths', input_death_df)
checkpoint.write('age_pop', input_age_pop_df)
checkpoint.write('age_death', input_age_death_df)

2020-05-04 17:30:10.470 | DEBUG    | covid_model_deaths.deaths_io.inputs:load:40 - Loading full_data.csv from /ihme/covid-19/model-inputs/2020_05_03.24.
2020-05-04 17:30:10.512 | DEBUG    | covid_model_deaths.deaths_io.inputs:load:40 - Loading deaths.csv from /ihme/covid-19/model-inputs/2020_05_03.24.
2020-05-04 17:30:10.533 | DEBUG    | covid_model_deaths.deaths_io.inputs:load:40 - Loading age_pop.csv from /ihme/covid-19/model-inputs/2020_05_03.24.
2020-05-04 17:30:10.541 | DEBUG    | covid_model_deaths.deaths_io.inputs:load:40 - Loading age_death.csv from /ihme/covid-19/model-inputs/2020_05_03.24.
2020-05-04 17:30:10.549 | DEBUG    | covid_model_deaths.deaths_io.inputs:load:40 - Loading us_pops.csv from /ihme/covid-19/model-inputs/2020_05_03.24.


## combine back-casted death rates with cases for abie (using model dataset, i.e. admin1 and below)

In [6]:
%%time
full_df = checkpoint.load('full_data')
death_df = checkpoint.load('deaths')
age_pop_df = checkpoint.load('age_pop')
age_death_df = checkpoint.load('age_death')

backcast_location_ids = runner.get_backcast_location_ids(full_df)
cases_and_backcast_deaths_df = runner.make_cases_and_backcast_deaths(full_df, death_df,
                                                                     age_pop_df, age_death_df,
                                                                     backcast_location_ids)

cases_and_backcast_deaths_df.to_csv(f'{OUTPUT_DIR}/backcast_for_case_to_death.csv', index=False)
checkpoint.write('cases_and_backcast_deaths', cases_and_backcast_deaths_df)

2020-05-04 18:00:14.447 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:30 - Loading full_data from in memory cache.
2020-05-04 18:00:14.451 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:30 - Loading deaths from in memory cache.
2020-05-04 18:00:14.452 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:30 - Loading age_pop from in memory cache.
2020-05-04 18:00:14.453 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:30 - Loading age_death from in memory cache.
100%|██████████| 178/178 [01:44<00:00,  1.70it/s]


CPU times: user 2.02 s, sys: 862 ms, total: 2.88 s
Wall time: 1min 45s


## Impute death thresholds.

In [4]:
%%time
cases_and_backcast_deaths_df = checkpoint.load('cases_and_backcast_deaths')
loc_df = checkpoint.load('location')
threshold_dates = runner.impute_death_threshold(cases_and_backcast_deaths_df,
                                                loc_df)
threshold_dates.to_csv(f'{OUTPUT_DIR}/threshold_dates.csv', index=False)
checkpoint.write('threshold_dates', threshold_dates)

2020-05-04 17:30:15.907 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:32 - Reading cases_and_backcast_deaths from checkpoint dir /ihme/covid-19/deaths/dev/2020_05_03_US_boundary/checkpoint.


FileNotFoundError: [Errno 2] No such file or directory: '/ihme/covid-19/deaths/dev/2020_05_03_US_boundary/checkpoint/cases_and_backcast_deaths.pkl'

## Make last day data

In [5]:
full_df = checkpoint.load('full_data')
threshold_dates = checkpoint.load('threshold_dates')

date_mean_df = runner.make_date_mean_df(threshold_dates)
last_day_df = runner.make_last_day_df(full_df,date_mean_df)
last_day_df.to_csv(f'{OUTPUT_DIR}/last_day.csv', index=False)

checkpoint.write('date_mean', date_mean_df)
checkpoint.write('last_day', last_day_df)

2020-05-04 17:30:16.009 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:30 - Loading full_data from in memory cache.
2020-05-04 17:30:16.010 | INFO     | covid_model_deaths.deaths_io.checkpoint:load:32 - Reading threshold_dates from checkpoint dir /ihme/covid-19/deaths/dev/2020_05_03_US_boundary/checkpoint.


FileNotFoundError: [Errno 2] No such file or directory: '/ihme/covid-19/deaths/dev/2020_05_03_US_boundary/checkpoint/threshold_dates.pkl'

## Get leading indicator

In [ ]:
full_df = checkpoint.load('full_data')
loc_df = checkpoint.load('location')

dcr_df, dhr_df, leading_indicator_df = runner.make_leading_indicator(
    full_df.loc[full_df[COLUMNS.location_id].isin(loc_df[COLUMNS.location_id].to_list())]
)
dcr_df.to_csv(f'{OUTPUT_DIR}/lagged_death_to_case_ratios.csv', index=False)
dhr_df.to_csv(f'{OUTPUT_DIR}/lagged_death_to_hosp_ratios.csv', index=False)
leading_indicator_df.to_csv(f'{OUTPUT_DIR}/leading_indicator.csv', index=False)
leading_indicator_df = leading_indicator_df[[COLUMNS.location_id, COLUMNS.date, COLUMNS.ln_age_death_rate]]
leading_indicator_df = leading_indicator_df.loc[~leading_indicator_df[COLUMNS.ln_age_death_rate].isnull()]

checkpoint.write('leading_indicator', leading_indicator_df)

## Submit models

In [ ]:
full_df = checkpoint.load('full_data')
death_df = checkpoint.load('deaths')
age_pop_df = checkpoint.load('age_pop')
age_death_df = checkpoint.load('age_death')
date_mean_df = checkpoint.load('date_mean')
last_day_df = checkpoint.load('last_day')
leading_indicator_df = checkpoint.load('leading_indicator')
loc_df = checkpoint.load('location')

submodel_dict = runner.submit_models(full_df, death_df, age_pop_df, age_death_df, date_mean_df, leading_indicator_df,
                                     loc_df, r0_locs,
                                     PEAK_FILE, OUTPUT_DIR, 'best',  # DATA_VERSION, 
                                     R0_FILE, CODE_DIR, verbose=False)

checkpoint.write('submodel_dict', submodel_dict)

## compile draws

In [ ]:
full_df = checkpoint.load('full_data')
age_pop_df = checkpoint.load('age_pop')
threshold_dates = checkpoint.load('threshold_dates')
submodel_dict = checkpoint.load('submodel_dict')
loc_df = checkpoint.load('location')

obs_df = full_df[full_df.location_id.isin(loc_df.location_id)]

draw_dfs, past_draw_dfs, models_used, days, ensemble_draws_dfs = runner.compile_draws(loc_df,
                                                                                      submodel_dict,
                                                                                      obs_df,
                                                                                      threshold_dates,
                                                                                      age_pop_df)

if 'location' not in models_used:
    raise ValueError('No location-specific draws used, must be using wrong tag')
draw_df = pd.concat(draw_dfs)
model_type_df = pd.DataFrame({'location': loc_df['Location'].tolist(),
                              'model_used': models_used})

# write
draw_df.to_csv(f'{OUTPUT_DIR}/state_data.csv', index=False)
model_type_df.to_csv(f'{OUTPUT_DIR}/state_models_used.csv', index=False)
ensemble_plot_path = runner.make_and_save_draw_plots(OUTPUT_DIR, loc_df,
                                                     ensemble_draws_dfs, days, models_used, age_pop_df)
print(ensemble_plot_path)
checkpoint.write('draw_data', draw_df)


## total US deaths in this run

In [ ]:
draw_df = checkpoint.load('draw_data')

runner.display_total_deaths(draw_df)

## combine with previous predictions

In [ ]:
avg_df = runner.average_draws(raw_draw_path, yesterday_draw_path, before_yesterday_draw_path)
avg_df.to_csv(average_draw_path, index=False)
compare_average_plot_path = runner.make_and_save_compare_average_plots(OUTPUT_DIR,
                                                                       raw_draw_path,
                                                                       average_draw_path,
                                                                       yesterday_draw_path,
                                                                       before_yesterday_draw_path,
                                                                       'United States of America')

In [ ]:
compare_to_previous_plot_path = runner.make_and_save_compare_to_previous_plots(OUTPUT_DIR,
                                                                               raw_draw_path,
                                                                               yesterday_draw_path,
                                                                               label='United States')


In [ ]:
viz_dir = runner.send_plots_to_diagnostics(DATESTAMP_LABEL,
                                           f'{OUTPUT_DIR}/ensemble_plot.pdf',
                                           #compare_average_plot_path,
                                           compare_to_previous_plot_path)
print(viz_dir)

## store deaths with smoothed past

In [ ]:
runner.smooth_data(OUTPUT_DIR, raw_draw_path)

## store point estimates, and peaks derived from them

In [ ]:
loc_df = checkpoint.load('location')
submodel_dict = checkpoint.load('submodel_dict')
draw_df = checkpoint.load('draw_data')
age_pop_df = checkpoint.load('age_pop')

runner.save_points_and_peaks(loc_df, submodel_dict, draw_df, age_pop_df, OUTPUT_DIR)